# Mapping Terms Application

This notebook explores the [Chronicling America API](https://chroniclingamerica.loc.gov/about/api/) for creating a map of the frequency certain terms appear in newspapers for a given time.

In [87]:
import pandas as pd
import requests
from geopy import geocoders
from geopy.extra.rate_limiter import RateLimiter
import plotly.offline as pyo
import plotly.graph_objs as go
import time

pyo.init_notebook_mode(connected=True)
searchTerm = "free+love"
beginYear = 1850
endYear = 1852
pg=1
endpg=2 #too much data? I'm just grabbing the first few pages right now
stop = False
allData = []
locs = {}
pd.set_option('display.max_colwidth', -1)
while not stop:
#http://chroniclingamerica.loc.gov/search/pages/results/?andtext={searchTerms}&page={startPage?}&ortext={chronam:booleanOrText?}&year={chronam:year?}&date1={chronam:date?}&date2={chronam:date?}&phrasetext={chronam:phraseText?}&proxText={chronam:proxText?}&proximityValue={chronam:proximityValue?}&format=json"
    url = "http://chroniclingamerica.loc.gov/search/pages/results/?phrasetext={0}&dateFilterType=yearRange&date1={1}&date2={2}&page={3}&format=json".format(searchTerm,beginYear,endYear,pg)
  
    call = requests.get(url)
    data = call.json()
    df = pd.DataFrame(data["items"])
    nf=pd.DataFrame()
    americanPanda=pd.DataFrame()
    #only want city and date for this application
    nf['date']=pd.to_datetime(df['date'])
    nf['citystate']=df['city'].map(lambda a:a[0])+', '+df['state'].map(lambda a: a[0])
    americanPanda['date']=pd.to_datetime(df['date'])
    americanPanda['location']=df['county'].map(lambda a:a[0])+', '+df['city'].map(lambda a: a[0])+', '+df['state'].map(lambda a: a[0])
    print(len(americanPanda.columns))
    loopCount=len(americanPanda.columns)
    secondPanda=pd.DataFrame()
    pdfList=[]
    textList=[]
    for count, url in enumerate(df['url'], start=1):
        print(count)
        call = requests.get(url)
        data = call.json()
        forframe = pd.DataFrame(data)
        forframe.index= [0,1,2]
        forpanda=pd.DataFrame()
        pdfframe=forframe['pdf']
        textframe=forframe['text']
        pdfList.append((pdfframe[0]))
        textList.append((textframe[0]))
    americanPanda['pdf']=pdfList
    americanPanda['text']=textList
    americanPanda['keywords']=searchTerm
    print(americanPanda)
    stop=True     

  


2
1
2
        date                         location  \
0 1851-03-04  Orleans, New Orleans, Louisiana   
1 1851-03-01  New York, New York, New York      

                                                                            pdf  \
0  https://chroniclingamerica.loc.gov/lccn/sn82015378/1851-03-04/ed-1/seq-4.pdf   
1  https://chroniclingamerica.loc.gov/lccn/sn83030313/1851-03-01/ed-1/seq-3.pdf   

                                                                               text  \
0  https://chroniclingamerica.loc.gov/lccn/sn82015378/1851-03-04/ed-1/seq-4/ocr.txt   
1  https://chroniclingamerica.loc.gov/lccn/sn83030313/1851-03-01/ed-1/seq-3/ocr.txt   

    keywords  
0  free+love  
1  free+love  
